In [22]:
import os
import pandas as pd
import numpy as np
import pathlib
import xarray as xr
import rioxarray

In [23]:
read_files_path = [
    r"./../data_global_surge/America/field_2024-09-29_12_00_00.f0.nc",
    r"./../data_global_surge/WNP/field_2024-09-29_12_00_00.f0.nc",
    r"./../data_global_surge/Oceania/field_2024-09-29_12_00_00.f0.nc",
    r"./../data_global_surge/IndiaOcean/field_2024-09-29_12_00_00.f0.nc",
]

In [24]:
# 打开所有文件并存储在一个列表中
datasets = [xr.open_dataset(fp) for fp in read_files_path]

In [25]:
# combined = xr.combine_by_coords(datasets, combine_attrs='override')

In [26]:
combined_ds = xr.combine_by_coords(datasets)

In [28]:
combined_ds.to_netcdf('combined_regions.nc')

In [20]:
combined_ds

<xarray.Dataset>
Dimensions:    (Time: 1, latitude: 3070, longitude: 6203)
Coordinates:
  * latitude   (latitude) float64 -55.0 -54.95 -54.9 -54.85 ... 59.9 59.95 60.0
  * longitude  (longitude) float64 20.0 20.05 20.1 20.15 ... 329.9 329.9 330.0
  * Time       (Time) datetime64[ns] 2024-09-29T12:00:00
Data variables:
    h          (Time, latitude, longitude) float32 nan nan nan ... 0.0 0.0 0.0
    maskLand   (latitude, longitude) float64 nan nan nan nan ... 0.0 0.0 0.0 0.0
    hMax       (latitude, longitude) float32 nan nan nan nan ... 0.0 0.0 0.0 0.0

In [15]:
for ds in datasets:
    print(ds.dims)

Frozen(SortedKeysDict({'longitude': 2401, 'latitude': 1201, 'Time': 1}))
Frozen(SortedKeysDict({'longitude': 901, 'latitude': 1001, 'Time': 1}))
Frozen(SortedKeysDict({'longitude': 2201, 'latitude': 1161, 'Time': 1}))
Frozen(SortedKeysDict({'longitude': 1601, 'latitude': 1361, 'Time': 1}))


In [12]:
# 使用 concat 在 'lat' 和 'lon' 维度上合并
combined = xr.concat(datasets, dim='lat', join='outer')
combined = xr.concat([combined], dim='lon', join='outer')

In [13]:
combined.to_netcdf(r'./../data_global_surge/combined_regions.nc')

In [43]:
# TODO:[-] 24-09-23 注意需要基于 maskland为掩码文件，将h与hmax陆地部分掩码掉
data['h']=data['h'].where(data['maskLand']==0,np.nan)
data['hMax']=data['hMax'].where(data['maskLand']==0,np.nan)
data

<xarray.Dataset>
Dimensions:    (Time: 1, latitude: 1001, longitude: 901)
Coordinates:
  * longitude  (longitude) float64 100.0 100.0 100.1 100.2 ... 144.9 144.9 145.0
  * latitude   (latitude) float64 5.0 5.05 5.1 5.15 ... 54.85 54.9 54.95 55.0
  * Time       (Time) datetime64[ns] 2024-09-29T12:00:00
Data variables:
    h          (Time, latitude, longitude) float32 nan nan ... -0.08421 -0.08543
    maskLand   (latitude, longitude) float64 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
    hMax       (latitude, longitude) float32 nan nan nan ... 0.005075 0.005922

In [44]:
data_standard=data.sortby('latitude',ascending=False)

In [45]:
first_time = data_standard["Time"].values[0]

In [46]:
first_dr = data_standard.sel(Time=first_time)["h"]
# 设置空间坐标系与分辨率
first_dr.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)
first_dr.rio.write_crs("EPSG:4326", inplace=True)

<xarray.DataArray 'h' (latitude: 1001, longitude: 901)>
array([[        nan,         nan,         nan, ..., -0.08420031,
        -0.08420587, -0.08542679],
       [        nan,         nan,         nan, ..., -0.08532447,
        -0.08577849, -0.08592574],
       [        nan,         nan,         nan, ..., -0.08697399,
        -0.087302  , -0.0877342 ],
       ...,
       [        nan,         nan,         nan, ...,  0.04560785,
         0.04545784,  0.04514027],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]], dtype=float32)
Coordinates:
  * longitude    (longitude) float64 100.0 100.0 100.1 ... 144.9 144.9 145.0
  * latitude     (latitude) float64 55.0 54.95 54.9 54.85 ... 5.15 5.1 5.05 5.0
    Time         datetime64[ns] 2024-09-29T12:00:00
    spatial_ref  int32 0
Attributes:
    units:         m
    long_name:     storm surge
    grid_mapping:  spatial_ref

In [47]:
out_put_file_path: str = (
    r"./../data_global_surge/field_2024-09-29_12_00_00.f0_WNP_standard_deflate.tif"
)
# first_dr.rio.to_raster(out_put_file_path,diver='GTiff',compress='deflate')
first_dr.rio.to_raster(out_put_file_path,diver='GTiff',compress='deflate')